In [ ]:
import torch
import model
import helper

In [ ]:
args = helper.Args()
helper.fix_seed(0)
device = torch.device("cuda:%d"%args.gpu_id if torch.cuda.is_available() else "cpu")

In [ ]:
my_model = model.ERM(args.num_classes, args)
my_model.to(device)

In [ ]:
train_dataloader = helper.get_ERM_dataloader(args, 'train')

In [ ]:
for step in range(args.num_steps):   
    x, y = next(train_dataloader)
    mini_batches = [x.to(device), y.to(device)]
    step_vals = my_model.update(mini_batches)
    if (step+1) % 20 == 0:
        log_str = "Step %d " % (step+1)
        for k, v in step_vals.items():
            log_str = log_str + "%s: %.4f, " % (k, v)
        print(log_str)

    if (step+1) % 100 == 0:        
        test_dataloader = helper.get_ERM_dataloader(args, 'test')        
        accuracy = helper.test(my_model, test_dataloader, device)
        print("ite: %d, test accuracy: %.4f" % (step+1, accuracy))